# 基于BM25的检索方法

---

核心思想：借助n-gram特征进行检索

In [1]:
import os
import re
import time
from tqdm import tqdm
import random
import sys
import pprint

sys.path.insert(0, "/home/team55/notespace/zengbin")
random.seed(1234)
from jddc.config import BM25Config
from jddc.utils import write_file, read_file, save_to_pkl, read_from_pkl, create_logger
from jddc.utils import insure_folder_exists
from jddc.bm25 import *

In [ ]:
print(str(conf))

## 1 - 准备数据
---

In [ ]:
all_sessions = read_from_pkl(conf.pkl_sessions)
# sessions = random.sample(all_sessions, 1000000)
# sessions = all_sessions

In [ ]:
left, right = random.sample(all_sessions[:400000], 100000),  random.sample(all_sessions[400000:], 200000)

In [ ]:
data_q = []
data_a = []

# 单轮数据集
for sess in left:
    q, a = sess.qaqaq_a
    if 2 < len(a) < 100:
        q = re.sub("(\t)| ", "", q)
        q_split = q.split("<s>")
        if len(q_split) == 5:
            q_merged = "".join([q_split[0], q_split[2], q_split[4]])
            data_q.append(q_merged.replace("\t", ""))
            data_a.append(a.replace("\t", " "))
            
# 多轮数据集
for sess in right:
    qas = sess.qas_merged
    if qas[0][0] == "1":
        qas = qas[1:]
    if qas[-1][0] == "0":
        qas = qas[:-1]
    assert len(qas) % 2 == 0
    for i in range(0, len(qas), 2):
        # 随机判断是否要新增QA
        if random.choice([True, False]):
            data_q.append(qas[i][1].replace("\t", ""))
            data_a.append(qas[i + 1][1].replace("\t", " "))
questions, answers = data_q, data_a

In [ ]:
questions, answers = create_dataset_for_bm25_01(sessions)

In [ ]:
len(questions)

## 2 - 构建/加载 bm25模型
---

In [ ]:
questions_tokens = []
for q in tqdm(questions, desc="cut"):
    # q_tokens = n_grams(q, conf.n)
    q_tokens = jieba_tokenize(q, for_search=False)
    questions_tokens.append(q_tokens)

In [7]:
data = read_file(os.path.join(conf.base_path, "seq2seq/train.tsv"))
questions_tokens = [x.split("\t")[0].split(" ") for x in data]
answers = [x.split("\t")[1].strip("\n").replace(" ", "") for x in data]

In [8]:
answers[0]

'亲，可以提供一下您的单位的信息吗'

In [9]:
model = bm25.BM25(questions_tokens)
average_idf = sum(float(val) for val in
                  model.idf.values()) / len(model.idf)
data = [model, answers, average_idf]
save_to_pkl(file=conf.pkl_bm25, data=data)

In [ ]:
model, answers, average_idf = create_bm25_model(questions, answers)

In [ ]:
model, answers, average_idf = load_bm25_model()

## 3 - 验证模型效果
----

预测时，首先根据切词后的句子长度进行筛选，仅保留长度相近的question，然后逐个计算相似度。

In [10]:
run_prediction(conf.file_test_q, "/home/team55/notespace/zengbin/answers/bm25_answers008.txt")

<BM25Config | model:/home/team55/notespace/data/BM25/bm25_3.model.small.pkl; n:3; top:5; random_num:1234>



run_prediction:   0%|                                                       | 0/111 [00:00<?, ?it/s]

load model from /home/team55/notespace/data/BM25/bm25_3.model.small.pkl success.
start predicting ...
question: 其他问题<q>我直接解绑了实名认证的那个账号，需要把手机号也解绑了 是的 金融客服说这个要找你们解决 他们不管



run_prediction:   1%|▍                                              | 1/111 [00:05<10:21,  5.65s/it]

['亲爱的，和您确认下，您是要解绑您正在沟通的这个账号的手机号么?#E-s[数字x]',
 '[USERID_10316267]亲亲您是要换绑一个手机号在这个账号上是吗',
 '亲，妹子可以帮您申请修改/解绑手机号的哦您之前的手机号码是多少呢?',
 '亲爱的，请问下是这个账号需要换绑的吗?#E-s[数字x]',
 '现在咨询的账户吗']
question: ok<q>新电话号码+[电话x]


...

['请问还有什么关于京准通的问题是我可以为您服务的呢?',
 '信用卡[数字x]-[数字x]5工作日到账',
 '您理解为赠送不收取佣金就可以了',
 '这边会帮您跟进至收到退款的呢',
 '您登入钱包哦']
question: 是不是直接就可以说<q>好



run_prediction:  99%|████████████████████████████████████████████▌| 110/111 [07:06<00:02,  2.68s/it]

['您好，宁波[地址x]，广州[地址x]，其他保税区没有的呢#E-s[数字x]#E-s[数字x]质量您可以放心的哦，都可以保证[数字x]%海外原装正品的哦',
 '这个不清楚的您放心，京东商城销售的都是正品真品!保证质量，请您放心购买使用呦~',
 '把新货给您拿过来的呢然后再把旧的取走',
 '是的哦每个订单都需要运费的哦',
 '亲是纯[姓名x]']
question: 好<q>谢谢!没了



run_prediction: 100%|█████████████████████████████████████████████| 111/111 [07:08<00:00,  2.29s/it]


['好的呢，亲爱的，这边小红人联系您的销售经理给您开通哦#E-s[数字x]',
 '这个还需要您耐心等待呢采购到货会尽快给您发出的呢',
 '亲，可以提供一下您的单位的信息吗',
 '请问还有其他还可以帮到您的吗?',
 '请您耐心等待退款呢']
